# Supervised Classification Model For Wine Dataset

Exploratory data analysis part can be found in Supervised-Regression-Modelling part. This is the continuos version for white wine data for classification.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df = pd.read_csv('winequality-white.csv', delimiter=';')
print(df.shape)
print(df.info())
print(df.head(2))

(4898, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB
None
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.

In [2]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [3]:
df.drop(['density'],axis=1,inplace=True)

# Library

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

# X and Y Data

In [5]:
X = np.array(df.drop('quality', axis=1))
y = np.array(df['quality'])

Since the data is already clean but still the outlier prevails, in order to fit well with the model without omitting the outlier, we choose the Robust scaler for scalling the data.

# Feature Selection

In [6]:
from sklearn.feature_selection import mutual_info_regression, SelectKBest
target_relation = mutual_info_regression(X, y)
print(target_relation)
k_select = SelectKBest(k=6)
X_feature = k_select.fit_transform(X,y)
print(X_feature.shape)

[0.03089078 0.04599747 0.05607352 0.09332027 0.05115644 0.06722981
 0.07046847 0.02891935 0.0281884  0.1645919 ]
(4898, 6)


# Data Balancing

In [7]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(k_neighbors=4)
X_train_smote,y_train_smote = smote.fit_sample(X_feature,y)
from collections import Counter
print('Before SMOTE:', Counter(y))
print('After SMOTE:', Counter(y_train_smote))

Before SMOTE: Counter({6: 2198, 5: 1457, 7: 880, 8: 175, 4: 163, 3: 20, 9: 5})
After SMOTE: Counter({6: 2198, 5: 2198, 7: 2198, 8: 2198, 4: 2198, 3: 2198, 9: 2198})


# Train Test Split

In [8]:
strat = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in strat.split(X_train_smote, y_train_smote):
    X_train, X_test = X_train_smote[train_index], X_train_smote[test_index]
    y_train, y_test = y_train_smote[train_index], y_train_smote[test_index]
    
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12308, 6) (3078, 6) (12308,) (3078,)


In [9]:
np.unique(y_train, return_counts=True)

(array([3, 4, 5, 6, 7, 8, 9]),
 array([1759, 1759, 1758, 1758, 1758, 1758, 1758]))

Stratified Shuffle Split helps to split the data according to the size-proportion of the target data. 

# Model Selection

In [ ]:
para_log = {'C': [0.1, 1, 10, 100]}
grid_log = GridSearchCV(LogisticRegression(max_iter=10000,random_state=42), param_grid= para_log, cv=4)
grid_log.fit(X_train, y_train)
print('Logistic Regression')
print(f'Cross_validation_score: {grid_log.best_score_}')
print(grid_log.best_params_)
print('---------')
para_svc = {'kernel': ['rbf'],
           'C': [100, 150, 200]}
grid_svc = GridSearchCV(SVC(), param_grid=para_svc, cv=3)
grid_svc.fit(X_train, y_train)
print('SVC')
print(f'Cross_validation_score: {grid_svc.best_score_}')
print(f'Best parameter : {grid_svc.best_params_}')
print('---------')
para_dt = {'max_depth': range(16,25)}
grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=para_dt, cv=3)
grid_dt.fit(X_train, y_train)
print('Decision Tree')
print(f'Cross_validation_score: {grid_dt.best_score_}')
print(f'Best parameter : {grid_dt.best_params_}')
print('---------')
para_rf = {'n_estimators': [300],
          'max_depth': range(20,30)}
# Cross-validation score is not increasing upon increasing the max depth from max_depth=21.
# Checking for features
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=para_rf, cv=3)
grid_rf.fit(X_train, y_train)
print('Random Forest')
print(f'Cross_validation_score: {grid_rf.best_score_}')
print(f'Best parameter : {grid_rf.best_params_}')
print('---------')

In [ ]:
para_ad = {'n_estimators': [50, 100, 200],
          'learning_rate': [0.01,0.1,1,10]}
grid_ad = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid=para_ad, cv=4)
grid_ad.fit(X_train, y_train)
print('AdaBoost-Classification')
print(f'Cross_validation_score: {grid_ad.best_score_}')
print(f'Best parameter : {grid_ad.best_params_}')

In [ ]:
para_gb = {'max_depth': [3,4,5,6,7]}
grid_gb = GridSearchCV(GradientBoostingClassifier(n_estimators=300,random_state=42), param_grid=para_gb, cv=3)
grid_gb.fit(X_train_smote, y_train_smote)
print('GradientBoost-Regressor')
print(f'Cross_validation_score: {grid_gb.best_score_}')
print(f'Best parameter : {grid_gb.best_params_}')

On model selection We have found that the best cross validation score is given by Random forest and Gradient Boosting classification. So, we select these models for our modelling purpose.

# Gradient Boosting Classification

In [ ]:
gb = GradientBoostingClassifier(n_estimators=300,random_state=42, learning_rate= 0.1, max_depth= 5, )
gb.fit(X_train_smote,y_train_smote)
pred_gb = gb.predict(X_test)
score_gb = accuracy_score(y_test,pred_gb)
print(f'Accuracy score: {score_gb}')
print(classification_report(y_test,pred_gb))
con_gb = confusion_matrix(y_test,pred_gb)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(con_gb,annot=True, fmt = '.3g', xticklabels=range(3,10),
            yticklabels=range(3,10))
plt.title("Confusion matrix of Gradient Boosting");

##  Explanation of the result

Although our model achieve approximately 0.79 in our cross-validation score, this model achive approximately 0.97 while introducing alien data. From Confusion matrix we found that most of the data were truely predicted. Apart from that, our model for test data is mostly misclassifying between quality level 5 followed by 4.

# Random forest

In [ ]:
rf_c = RandomForestClassifier(max_depth=26, n_estimators=300,random_state=42)
rf_c.fit(X_train,y_train)
pred_rf = rf_c.predict(X_test)
score_rf = accuracy_score(y_test,pred_rf)
print(f'Accuracy score: {score_rf}')
print(classification_report(y_test,pred_rf))
con_rf = confusion_matrix(y_test,pred_rf)
plt.figure(figsize=(8,8))
sns.heatmap(con_rf,annot=True, fmt = '.3g', xticklabels=range(3,10),
            yticklabels=range(3,10))
plt.title("Confusion matrix of Random Forest");

## Ilustrating the feature selection for Random Forest model

In [ ]:
selected_columns = df.iloc[:,np.argwhere(target_relation>0.043).flatten()].columns
selected_columns
plt.barh(range(1,6),selected_columns)
plt.yticks(range(1,6), df.columns)
plt.xlabel('Feature importance score')
plt.ylabel('Features');

# Conclusion

Our classification task give us slightly better model. However in comparing between Gradient Boosting and Random forest we find that, Gradient boost model is nicely fitting with the accuracy of approximately 0.97. If consider with the accuracy score to the cross-validation score our cross-validation score is lower than our accuracy score. This might be the chance of overfitting for the future reference. But the Random forest classifier is generalizing well with the cross-validating score of 0.82 and accuracy score of 0.87.

Note:
do data analysis, show the relationship between each features, or may be selected features before going to modeling. clean data, see normalize (model selection, cross validation score of those models,
For the classification, make your own class, like good, okay and bad...
in model selection, no need to do grid search, but use cross validation